In [84]:
%matplotlib qt
import matplotlib
import datetime
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as mpl
from pathlib import Path
import pandas as pd
from pylibrary.tools import cprint as CP
cprint = CP.cprint
# from pylibrary import Params
# base = "/Users/pbmanis/Desktop/Python/VCN-SBEM-Data"
# cell = 2
# iv_simpath = f"VCN_Cells/VCN_c{cell:02d}/Simulations/IV/"
# simdir = Path(base, iv_simpath)
# files = list(simdir.glob('*'))

def get_timestamp(file):
    mtime = file.stat().st_mtime
    timestamp_str = datetime.datetime.fromtimestamp(mtime).strftime(
            "%Y-%m-%d.%H-%M-%S"
        )
    return timestamp_str

def switch_timestamp(timestamp):
    """
    Switch the time stamp to different format
    Here the initial value is a string, which we convert to a datetime
      
    """
    times = datetime.datetime.strptime(timestamp,"%a %b %d %H:%M:%S %Y") 
    times_str = datetime.datetime.strftime(times,
            "%Y-%m-%d.%H-%M-%S"
        )
    return str(times_str)

    
def find_files(files):
    for i, f in enumerate(files):
        if f.is_file():  # skips the files (and index files with .pkl extension)
            continue
        timestamp_str = get_timestamp(f)
        fn = Path(base, iv_simpath, f)
        cprint('y', f"Directory:  {str(fn):s}")
        if fn.name.startswith('initAN') or fn.name.startswith('testAN') or len(list(fn.glob('*'))) == 0:
            cprint('r', f"Removing empty directory: {fn.name:s}")
            fn.rmdir()  # removes the empty dir, and init/test directories that do not belong here
            continue
            
        for fns in f.glob('*'):  # now look at the directory, get the timestamp from the first file
            if fns.is_file():
                print('     file: ', fns.name, get_timestamp(fns))
                d = pd.read_pickle(fns)
                if isinstance(d['runInfo'], dict):
                    ts = d['runInfo']['runTime']  # this is the time we want
                else:
                    ts = d['runInfo'].runTime  # this is the time we want
#                 print('   File Internal Run time: ', ts)
                times = switch_timestamp(ts)
                dirname = str(f)
                newdirname = Path(f.parent, f.name[:10] +  times)
#                 print('   olddirname: ', dirname, '\n  newdir name: ', newdirname)
                # look for the matching .pkl file in the directory above
                pklname = dirname + '.pkl'
                pklname = Path(pklname)
                if pklname.is_file():
                    print(f"Matching Index file found: {str(pklname):s}")
                else:
                    print(f"Index NOT found: {str(pklname):s}")
                
                break # that is all folks.
 
cells = [2, 5, 6, 9, 10, 11, 13, 17, 30]
for cell in cells:
    iv_simpath = f"VCN_Cells/VCN_c{cell:02d}/Simulations/AN/"
    simdir = Path(base, iv_simpath)
    files = list(simdir.glob('*'))
    find_files(files)

Directory:  /Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c02/Simulations/AN/runANSingles-all-2020-11-04.12-35-41
     file:  AN_Result_VCN_c02_inp=self_XM13A_nacncoop_II_HF=VCN_c02_Full_soma=1.462_dend=1.465_normal_Syn002_multisite_005_tonepip_050dB_16000.0_HS.p 2020-11-04.12-40-51
Matching Index file found: /Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c02/Simulations/AN/runANSingles-all-2020-11-04.12-35-41.pkl
Directory:  /Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c02/Simulations/AN/initAN-all-2020-09-01.21-31-54
Removing empty directory: initAN-all-2020-09-01.21-31-54
Directory:  /Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c02/Simulations/AN/runANPSTH-largestonly-2020-09-01.21-36-45
     file:  AN_Result_VCN_c02_inp=self_XM13A_nacncoop_II_soma=1.462_dend=1.465_normal_largestonly_multisite_020_SAM_020dB_16000.0_200.0_100_HS.p 2020-09-01.21-41-16
Matching Index file found: /Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c

KeyError: 'runInfo'